In [0]:
# Install pySpark
# This only needs to be done once per notebook (?)
# apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q http://apache.claz.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
# !tar xf spark-2.4.0-bin-hadoop2.7.tgz
# !ls -al /usr/lib/jvm/java-8-openjdk-amd64
# !pip install -q findspark

total 28
drwxr-xr-x 7 root root 4096 Feb  8 17:10 .
drwxr-xr-x 4 root root 4096 Feb  8 17:08 ..
lrwxrwxrwx 1 root root   22 Jan 14 21:02 ASSEMBLY_EXCEPTION -> jre/ASSEMBLY_EXCEPTION
drwxr-xr-x 2 root root 4096 Feb  8 17:10 bin
lrwxrwxrwx 1 root root   41 Jan 14 21:02 docs -> ../../../share/doc/openjdk-8-jre-headless
drwxr-xr-x 3 root root 4096 Feb  8 17:10 include
drwxr-xr-x 5 root root 4096 Feb  8 17:08 jre
drwxr-xr-x 3 root root 4096 Feb  8 17:10 lib
drwxr-xr-x 4 root root 4096 Feb  8 17:08 man
lrwxrwxrwx 1 root root   20 Jan 14 21:02 src.zip -> ../openjdk-8/src.zip
lrwxrwxrwx 1 root root   22 Jan 14 21:02 THIRD_PARTY_README -> jre/THIRD_PARTY_README


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()


In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
# file_id = 'REPLACE_WITH_YOUR_FILE_ID'
# downloaded = drive.CreateFile({'id': file_id})
# print('Downloaded content "{}"'.format(downloaded.GetContentString()))

from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
%%bigquery --project gbqthodlas df

SELECT count(*) FROM `publicdata.samples.github_timeline`

,f0_
0,6219749


In [0]:
project_id = 'gbqthodlas'

In [0]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

sample_count = 2000
row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `publicdata.samples.github_timeline`''').to_dataframe().total[0]

df = client.query('''
  SELECT
    *
  FROM
    `publicdata.samples.github_timeline`
  WHERE RAND() < %d/%d
''' % (sample_count, row_count)).to_dataframe()

print('Full dataset has %d rows' % row_count)

Full dataset has 6219749 rows


In [0]:
# list(df)
df.head(10)

[Row(hello='world'),
 Row(hello='world'),
 Row(hello='world'),
 Row(hello='world'),
 Row(hello='world'),
 Row(hello='world'),
 Row(hello='world'),
 Row(hello='world'),
 Row(hello='world'),
 Row(hello='world')]

In [34]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *


if __name__ == "__main__":
    sc = SparkContext(appName="CSV2Parquet")
    sqlContext = SQLContext(sc)
    
    schema = StructType([
            StructField("customer_id", IntegerType(), True),
            StructField("customer_name", StringType(), True),
            StructField("category", StringType(), True)])
    
    rdd = sc.textFile("\\\\lordabbett.com\\nas\\EnterprisePlatforms\\Qlik\\Hackathon\\qs_test_data\\customer.csv").map(lambda line: line.split(","))
    df = sqlContext.createDataFrame(rdd, schema)
    df.write.parquet('\\\\lordabbett.com\\nas\\EnterprisePlatforms\\Qlik\\Hackathon\\qs_test_data\\customer-parquet')

Py4JJavaError: ignored